# IMDB 영화 리뷰 감성 분석 - Conv1D
 - Conv1D + Conv1D + Dense
 - Conv1D + LSTM + Dense
 - Conv1D + Dense + Dense

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout

In [3]:
num_words=10000
(X_train, y_train),(_,_) = imdb.load_data(num_words=num_words)

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

17473536/17464789 [==============================] - 0s 0us/step


((20000,), (5000,), (20000,), (5000,))

In [4]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((20000, 500), (5000, 500))

 - case1: Conv1D + Conv1D

In [5]:
model1 = Sequential([
                     Embedding(num_words, 100, input_length=max_len),
                     Dropout(0.5),
                     Conv1D(64, 7, activation='relu'),
                     MaxPooling1D(7),
                     Conv1D(64, 5, activation='relu'),
                     MaxPooling1D(5),
                     GlobalMaxPooling1D(),
                     Dense(1, activation='sigmoid')
                     ])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

 - case2: Conv1D +LSTM

In [6]:
model2 = Sequential([
                     Embedding(num_words, 100, input_length=max_len),
                     Dropout(0.5),
                     Conv1D(64, 5, activation='relu'),
                     MaxPooling1D(5),
                     LSTM(100),
                     Dense(1, activation='sigmoid')
                     ])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

 - case3: Conv1D + Dense + Dense

In [7]:
model3 = Sequential([
                     Embedding(num_words, 100, input_length=max_len),
                     Dropout(0.5),
                     Conv1D(64, 5, activation='relu'),
                     MaxPooling1D(5),
                     GlobalMaxPooling1D(),
                     Dense(100, activation='relu'),
                     Dense(1, activation='sigmoid')
                     ])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [8]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path1 = 'best_imdb_conv1d.h5'
mc1 = ModelCheckpoint(model_path1, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [9]:
hist1 = model1.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=64, callbacks=[mc1,es1])

Epoch 1/30
244/250 [============================>.] - ETA: 0s - loss: 0.5617 - accuracy: 0.6733
Epoch 1: val_loss improved from inf to 0.36129, saving model to best_imdb_conv1d.h5
250/250 [==============================] - 14s 10ms/step - loss: 0.5565 - accuracy: 0.6778 - val_loss: 0.3613 - val_accuracy: 0.8480
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.2876 - accuracy: 0.8816
Epoch 2: val_loss improved from 0.36129 to 0.32399, saving model to best_imdb_conv1d.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2876 - accuracy: 0.8816 - val_loss: 0.3240 - val_accuracy: 0.8590
Epoch 3/30
249/250 [============================>.] - ETA: 0s - loss: 0.1941 - accuracy: 0.9256
Epoch 3: val_loss did not improve from 0.32399
250/250 [==============================] - 2s 9ms/step - loss: 0.1939 - accuracy: 0.9257 - val_loss: 0.3539 - val_accuracy: 0.8560
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.1307 - accuracy: 0.9533
Ep

In [10]:
best_model1 = load_model(model_path1)
best_model1.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.3081 - accuracy: 0.8684


[0.3081115186214447, 0.868399977684021]

In [11]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path2 = 'best_imdb_conv1d_lstm.h5'
mc2 = ModelCheckpoint(model_path2, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [12]:
hist2 = model2.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=64, callbacks=[mc2,es2])

Epoch 1/30
249/250 [============================>.] - ETA: 0s - loss: 0.4514 - accuracy: 0.7604
Epoch 1: val_loss improved from inf to 0.29500, saving model to best_imdb_conv1d_lstm.h5
250/250 [==============================] - 6s 15ms/step - loss: 0.4509 - accuracy: 0.7609 - val_loss: 0.2950 - val_accuracy: 0.8888
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.2171 - accuracy: 0.9138
Epoch 2: val_loss improved from 0.29500 to 0.25895, saving model to best_imdb_conv1d_lstm.h5
250/250 [==============================] - 3s 13ms/step - loss: 0.2171 - accuracy: 0.9138 - val_loss: 0.2589 - val_accuracy: 0.8963
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 0.1517 - accuracy: 0.9430
Epoch 3: val_loss did not improve from 0.25895
250/250 [==============================] - 3s 13ms/step - loss: 0.1515 - accuracy: 0.9432 - val_loss: 0.2801 - val_accuracy: 0.8928
Epoch 4/30
249/250 [============================>.] - ETA: 0s - loss: 0.0951 - accuracy

In [13]:
best_model2 = load_model(model_path2)
best_model2.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.2717 - accuracy: 0.8902


[0.2716953158378601, 0.8902000188827515]

In [14]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])
model_path3 = 'best_imdb_conv1d_dense2.h5'
mc3 = ModelCheckpoint(model_path3, verbose=1, save_best_only=True)
es3= EarlyStopping(patience=5)

In [15]:
hist3 = model3.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=64, callbacks=[mc3,es3])

Epoch 1/30
246/250 [============================>.] - ETA: 0s - loss: 0.5797 - accuracy: 0.6596
Epoch 1: val_loss improved from inf to 0.38756, saving model to best_imdb_conv1d_dense2.h5
250/250 [==============================] - 3s 9ms/step - loss: 0.5765 - accuracy: 0.6624 - val_loss: 0.3876 - val_accuracy: 0.8285
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.3096 - accuracy: 0.8682
Epoch 2: val_loss improved from 0.38756 to 0.29800, saving model to best_imdb_conv1d_dense2.h5
250/250 [==============================] - 2s 8ms/step - loss: 0.3096 - accuracy: 0.8682 - val_loss: 0.2980 - val_accuracy: 0.8802
Epoch 3/30
250/250 [==============================] - ETA: 0s - loss: 0.2058 - accuracy: 0.9215
Epoch 3: val_loss improved from 0.29800 to 0.28787, saving model to best_imdb_conv1d_dense2.h5
250/250 [==============================] - 2s 8ms/step - loss: 0.2058 - accuracy: 0.9215 - val_loss: 0.2879 - val_accuracy: 0.8867
Epoch 4/30
249/250 [==================

In [16]:
best_model3= load_model(model_path3)
best_model3.evaluate(X_test, y_test)

157/157 [==============================] - 1s 4ms/step - loss: 0.2874 - accuracy: 0.8824


[0.2874262034893036, 0.8823999762535095]